# First Data Science Project
## Melbourne Housing Prices Prediction
Here, we will go through a data challenge using data predicting housing prices in Melbourne, Australia. 

The data is from Kaggle and can be found [here](https://www.kaggle.com/anthonypino/melbourne-housing-market)

In [134]:
import numpy as np
import pandas as pd
import plotly.express as px
import sklearn
sklearn.set_config(transform_output="pandas")

## 1. Collection

In [182]:
full_data = pd.read_csv("./data/Melbourne_housing_FULL.csv")
full_data.shape

(34857, 21)

## 2. Cleaning

In [136]:

col_nan = full_data.columns[col_nan_b]
data_rmv_nan = full_data.dropna()
data_rmv_nan.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0
6,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra City Council,-37.8072,144.9941,Northern Metropolitan,4019.0
11,Abbotsford,124 Yarra St,3,h,1876000.0,S,Nelson,7/05/2016,2.5,3067.0,...,2.0,0.0,245.0,210.0,1910.0,Yarra City Council,-37.8024,144.9993,Northern Metropolitan,4019.0
14,Abbotsford,98 Charles St,2,h,1636000.0,S,Nelson,8/10/2016,2.5,3067.0,...,1.0,2.0,256.0,107.0,1890.0,Yarra City Council,-37.8060,144.9954,Northern Metropolitan,4019.0


Viewing the colomns with NAN

In [137]:
fig = px.scatter(data_rmv_nan, x='YearBuilt', y='Price')
fig.show()

In [138]:
# removing the outlier

def clean_outlier(outlier, df):
    
    for k,v in outlier.items():
        print(k, v)
        for vi in v:
            # print(vi)
            mask_vi = df.loc[:, k] == vi
            df.drop(df.loc[mask_vi, k].index, inplace=True)

    return df
    
outlier = {'YearBuilt': [1196]}
data = clean_outlier(outlier,data)
fig = px.scatter(data, x='YearBuilt', y='Price')
fig.show()

YearBuilt [1196]


/tmp/ipykernel_133468/3560282840.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [116]:
# removin NAN rows

print(data['CouncilArea'].unique())
for col in col_nan:
    if col == 'YearBuilt' or col=='Price': 
        continue
    print(col)
    fig = px.scatter(data_rmv_nan, x='YearBuilt', y='Price', size=col, title=col)
    # fig.show()
    

['Yarra City Council' 'Moonee Valley City Council'
 'Port Phillip City Council' 'Darebin City Council'
 'Hobsons Bay City Council' 'Stonnington City Council'
 'Boroondara City Council' 'Monash City Council' 'Glen Eira City Council'
 'Whitehorse City Council' 'Maribyrnong City Council'
 'Bayside City Council' 'Moreland City Council' 'Manningham City Council'
 'Melbourne City Council' 'Banyule City Council' 'Brimbank City Council'
 'Kingston City Council' 'Hume City Council' 'Knox City Council'
 'Melton City Council' 'Maroondah City Council'
 'Greater Dandenong City Council' 'Nillumbik Shire Council'
 'Whittlesea City Council' 'Frankston City Council'
 'Macedon Ranges Shire Council' 'Yarra Ranges Shire Council'
 'Wyndham City Council' 'Casey City Council' 'Cardinia Shire Council'
 'Mitchell Shire Council' 'Moorabool Shire Council']
Distance
Postcode
Bedroom2
Bathroom
Car
Landsize
BuildingArea
CouncilArea


TypeError: unsupported operand type(s) for /: 'str' and 'int'

## 3. Exploratory Analysis

## 4. Model Building

In [253]:
from sklearn.model_selection import train_test_split
from sklearn import tree
import category_encoders as ce
from sklearn.metrics import mean_squared_error


data = data_cln

# outlier = {'YearBuilt': [1196]}
# data = clean_outlier(outlier,data)
# print(data.shape)


## columns with integerim
str_col = data.select_dtypes(include='object')

y = data.loc[:, 'Price']
X = data.drop('Price', axis=1)

# mean encoding
enc = ce.target_encoder.TargetEncoder(cols=str_col).fit(X, y)
X_enc_m = enc.transform(X)
#-------------------------------------------------

# Drop values with str
X_d = X.drop(str_col, axis=1)
#-------------------------------------------------

# mix mean and ordinal
mean_col_list = []
mean_col = X.loc[:, mean_col_list]
ord_col = str_col.drop(mean_col_list, axis=1)

    ## just mean for Date
enc_m = ce.target_encoder.TargetEncoder(cols=mean_col).fit(X, y)
X_enc_m = enc_m.transform(X)
enc_O = ce.ordinal.OrdinalEncoder(cols=ord_col).fit(X_enc_m)
X_enc_OM = enc_O.transform(X_enc_m)
#-------------------------------------------------

# drop some
sel = ['CouncilArea', 'Postcode', 'Date', 'Address', 'SellerG'] #,
X_d_s = X.drop(sel, axis=1)

str_col = X_d_s.select_dtypes(include='object')
enc_O = ce.ordinal.OrdinalEncoder(cols=str_col).fit(X_d_s)
X_d_s_n = enc_O.transform(X_d_s)

# mean encoding
enc = ce.target_encoder.TargetEncoder(cols=str_col).fit(X_d_s, y)
X_d_s_n = enc.transform(X_d_s)
#-------------------------------------------------

X_n = X_d_s_n
X_train, X_test, y_train, y_test = train_test_split(X_n, y, test_size=0.2, random_state=42)


clf = tree.DecisionTreeRegressor()
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_pred_train = clf.predict(X_train)

RMSE = mean_squared_error(y_test, y_pred, squared=False)

RMSE_train = mean_squared_error(y_train, y_pred_train, squared=False)
X.shape, X_train.shape, X_test.shape, RMSE, RMSE_train, 



((27244, 12), (21795, 7), (5449, 7), 376110.25683993957, 265047.41296982474)

## 5. Iterating